## Basic Query with the LangChain Wrapper

In [26]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0)
# Run basic query with OpenAI wrapper

In [27]:
from openai import OpenAI
from IPython.display import Markdown
client = OpenAI()

def get_response(prompt_question):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo-16k",
        messages=[{"role": "system", "content": "You are a helpful research and programming assistant"},
                  {"role": "user", "content": prompt_question}]
    )
    
    return response.choices[0].message.content

output = get_response("Explain large language models in one sentence")
Markdown(output)

Large language models are advanced artificial intelligence systems that are trained on vast amounts of text data and can generate human-like responses to prompts and questions.

## Importing and defining Schema for the chat messages

In [28]:
# import schema for chat messages and ChatOpenAI in order to query chatmodels GPT-3.5-turbo or GPT-4

from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.chat_models import ChatOpenAI

In [29]:
chat = ChatOpenAI(model_name="gpt-4",temperature=0.3)
messages = [
    SystemMessage(content="You are an expert data scientist"),
    HumanMessage(content="Write a Python script that trains a neural network on simulated data ")
]
response=chat(messages)

print(response.content,end='\n')

Sure, here's a simple example of how you can train a neural network using Python and Keras. This script will generate a dataset using sklearn, then it will train a neural network on this data.

```python
from sklearn.datasets import make_classification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Generate a binary classification dataset.
X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=7)

# Define the neural network model.
model = Sequential()
model.add(Dense(25, input_dim=20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model.
opt = Adam(learning_rate=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

# Fit the model on the dataset.
model.fit(X, y, epochs=100, batch_size=10, verbose=0)

# Evaluate the model.
_, accuracy = model.evaluate(X, y, verbose=0)
print('Accuracy: %.2

## Import prompt and define PromptTemplate

In [30]:
from langchain import PromptTemplate

template = """
You are an expert data scientist with an expertise in building deep learning models. 
Explain the concept of {concept} in a couple of lines
"""

prompt = PromptTemplate(
    input_variables=["concept"],
    template=template,
)

In [31]:
output = get_response(prompt.format(concept="autoencoder"))
print(output)

An autoencoder is a type of artificial neural network that is trained to reconstruct its input data. It consists of an encoder network that encodes the input data into a lower-dimensional representation and a decoder network that reconstructs the original data from the encoded representation. The autoencoder is trained to minimize the difference between the original input and the output generated by the decoder, effectively learning to compress and decompress the data.


## Importing and defining the LLMChain with language model and prompt as arguments.

In [32]:

from langchain.chains import LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain only specifying the input variable.
print(chain.run("autoencoder"))

An autoencoder is a type of neural network that is trained to reconstruct its input data. It consists of an encoder network that compresses the input data into a lower-dimensional representation, and a decoder network that reconstructs the original input from this representation. Autoencoders are commonly used for dimensionality reduction, feature extraction, and anomaly detection.


### Second Prompt

In [33]:
second_prompt = PromptTemplate(
    input_variables=["ml_concept"],
    template="Turn the concept description of {ml_concept} and explain it to me like I'm five in 500 words",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

## Define a sequential chain using the two chains above


In [34]:
from langchain.chains import SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain, chain_two], verbose=True)

# Run the chain specifying only the input variable for the first chain.
explanation = overall_chain.run("self-attention")
print(explanation)



> Entering new SimpleSequentialChain chain...
Self-attention, also known as scaled dot-product attention, is a mechanism in deep learning models that allows the model to focus on different parts of the input sequence when making predictions. It computes attention weights for each element in the sequence by comparing it with all other elements, capturing the importance of each element in relation to others. This enables the model to effectively capture long-range dependencies and improve its ability to understand and generate coherent outputs.
Imagine you have a special power that allows you to pay attention to different things around you. Let's say you're in a classroom and the teacher is explaining a lesson. Normally, you would pay attention to the teacher's words and gestures to understand what they are saying. But sometimes, you might also want to pay attention to what your classmates are doing or what's written on the board.

In deep learning models, there is a similar concept ca

In [35]:
# Import utility for splitting up texts and split up the explanation given above into document chunks

from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap  = 0,
)

texts = text_splitter.create_documents([explanation])

In [36]:

# Individual text chunks can be accessed with "page_content"

texts[0].page_content


'Imagine you have a special power that allows you to pay attention to different things around you.'

## Import and instantiate OpenAI embeddings

In [39]:
from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

In [40]:

# Turn the first text chunk into a vector with the embedding

query_result = embeddings.embed_query(texts[0].page_content)
print(query_result)

[-0.008101751282596372, 0.002717129017224307, 0.028820459461437565, -0.02390176857761959, -0.021045340708388593, 0.006151567354502718, -0.026463587444766734, -0.04408890110172746, -0.011355261579149378, -0.019431394039774914, 0.018329812634706816, 0.009344233414536284, -0.006135555590770945, -0.0061771852451509585, 0.015050684447505569, -0.010836493472756278, 0.029870805085209184, 0.01676710361003481, -0.0008373944630363016, -0.008005683494173523, -0.017817449233806425, 0.007903211000257964, 0.0005531926999244641, -0.016062602769643525, -0.025695041412093552, -0.0016299571517768838, 0.012130211385992672, -0.008325910386905618, -0.00539903308556131, -0.01920083022997296, 0.04593340785485271, -0.01275785669179404, -0.005856957886757566, -0.0224799593484968, -0.016241930798149, -0.00017762609466894064, 0.011220765557770273, -0.024631886377336515, 0.02331255057345188, -0.0013617667449431837, 0.02712966005220176, 0.01488416676130811, -0.003999639394340407, 0.005943419548263947, -0.003314352

## Creating an agent to solve a simple quadratic equation

In [2]:
# Import Python REPL tool and instantiate Python agent

from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI

agent_executor = create_python_agent(
    llm=OpenAI(model_name="gpt-3.5-turbo-1106" ,temperature=0, max_tokens=1000),
    tool=PythonREPLTool(),
    verbose=True
)


In [ ]:
# Execute the Python agent

agent_executor.run("Find the roots (zeros) if the quadratic function 3 * x**2 + 2*x -1")     



> Entering new AgentExecutor chain...
I can use the quadratic formula to find the roots of the quadratic function.
Action: Python_REPL
Action Input: import cmath
Observation: 
Thought:I need to define the coefficients of the quadratic function and then use the quadratic formula to find the roots.
Action: Python_REPL
Action Input: a = 3, b = 2, c = -1
Observation: SyntaxError('cannot assign to literal', ('<string>', 1, 5, 'a = 3, b = 2, c = -1\n'))
Thought:I need to define the coefficients separately.
Action: Python_REPL
Action Input: a = 3
Observation: 
Thought:I need to define the other coefficients separately as well.
Action: Python_REPL
Action Input: b = 2
Observation: 
Thought:I can now use the quadratic formula to find the roots.
Action: Python_REPL
Action Input: root1 = (-b + cmath.sqrt(b**2 - 4*a*c)) / (2*a), root2 = (-b - cmath.sqrt(b**2 - 4*a*c)) / (2*a)
Observation: SyntaxError('cannot assign to operator', ('<string>', 1, 9, 'root1 = (-b + cmath.sqrt(b**2 - 4*a*c)) / (2*a),

'root1 = (0.3333333333333333+0j), root2 = (-1+0j)'

## LCEL example

### Without LCEL

In [3]:
from typing import List

import openai


prompt_template = "Tell me a short joke about {topic}"
client = openai.OpenAI()

def call_chat_model(messages: List[dict]) -> str:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo", 
        messages=messages,
    )
    return response.choices[0].message.content

def invoke_chain(topic: str) -> str:
    prompt_value = prompt_template.format(topic=topic)
    messages = [{"role": "user", "content": prompt_value}]
    return call_chat_model(messages)

invoke_chain("ice cream")

'Why did the ice cream go to therapy?\n\nBecause it had a rocky road!'

### With LCEL

In [4]:
from langchain_core.runnables import RunnablePassthrough


prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-3.5-turbo")
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

chain.invoke("ice cream")

'Why did the ice cream go to therapy? \n\nBecause it had a rocky road!'

## Callbacks using multiple handlers

In [9]:
from typing import Any, Dict, List, Union

from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.callbacks.base import BaseCallbackHandler
from langchain.schema import AgentAction
from langchain_openai import OpenAI


# First, define custom callback handler implementations
class MyCustomHandlerOne(BaseCallbackHandler):
    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        print(f"on_llm_start {serialized['name']}")

    def on_llm_new_token(self, token: str, **kwargs: Any) -> Any:
        print(f"on_new_token {token}")

    def on_llm_error(
        self, error: Union[Exception, KeyboardInterrupt], **kwargs: Any
    ) -> Any:
        """Run when LLM errors."""

    def on_chain_start(
        self, serialized: Dict[str, Any], inputs: Dict[str, Any], **kwargs: Any
    ) -> Any:
        print(f"on_chain_start {serialized['name']}")

    def on_tool_start(
        self, serialized: Dict[str, Any], input_str: str, **kwargs: Any
    ) -> Any:
        print(f"on_tool_start {serialized['name']}")

    def on_agent_action(self, action: AgentAction, **kwargs: Any) -> Any:
        print(f"on_agent_action {action}")


class MyCustomHandlerTwo(BaseCallbackHandler):
    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        print(f"on_llm_start (I'm the second handler!!) {serialized['second handler']}")


# Instantiate the handlers
handler1 = MyCustomHandlerOne()
handler2 = MyCustomHandlerTwo()

# Setup the agent. Only the `llm` will issue callbacks for handler2
llm = OpenAI(temperature=0, streaming=True, callbacks=[handler2])
tools = load_tools(["llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION)

# Callbacks for handler1 will be issued by every object involved in the
# Agent execution (llm, llmchain, tool, agent executor)
agent.run("What is 2 raised to the 0.235 power?", callbacks=[handler1])

Error in MyCustomHandlerOne.on_chain_start callback: KeyError('name')
Error in MyCustomHandlerOne.on_chain_start callback: KeyError('name')
Error in MyCustomHandlerOne.on_llm_start callback: KeyError('name')
Error in MyCustomHandlerTwo.on_llm_start callback: KeyError('second handler')


on_new_token  I
on_new_token  should
on_new_token  use
on_new_token  a
on_new_token  calculator
on_new_token  to
on_new_token  solve
on_new_token  this
on_new_token .
Action
on_new_token :
on_new_token  Calculator
on_new_token 
Action
on_new_token  Input
on_new_token :
on_new_token  
on_new_token 2


Error in MyCustomHandlerOne.on_chain_start callback: KeyError('name')
Error in MyCustomHandlerOne.on_chain_start callback: KeyError('name')
Error in MyCustomHandlerOne.on_llm_start callback: KeyError('name')
Error in MyCustomHandlerTwo.on_llm_start callback: KeyError('second handler')


on_new_token  ^
on_new_token  
on_new_token 0
on_new_token .
on_new_token 235
on_new_token 
on_agent_action tool='Calculator' tool_input='2 ^ 0.235' log=' I should use a calculator to solve this.\nAction: Calculator\nAction Input: 2 ^ 0.235'
on_tool_start Calculator
on_new_token ```text
on_new_token 

on_new_token 2
on_new_token  **
on_new_token  
on_new_token 0
on_new_token .
on_new_token 235
on_new_token 

on_new_token ```

on_new_token ...
on_new_token num
on_new_token expr
on_new_token .evaluate
on_new_token ("
on_new_token 2
on_new_token  **
on_new_token  
on_new_token 0


Error in MyCustomHandlerOne.on_chain_start callback: KeyError('name')
Error in MyCustomHandlerOne.on_llm_start callback: KeyError('name')
Error in MyCustomHandlerTwo.on_llm_start callback: KeyError('second handler')


on_new_token .
on_new_token 235
on_new_token ")
on_new_token ...

on_new_token 
on_new_token  I
on_new_token  now
on_new_token  know
on_new_token  the
on_new_token  final
on_new_token  answer
on_new_token .
Final Answer:
on_new_token  
on_new_token 1.
on_new_token 1769067372187674


'1.1769067372187674'